# Probability of wearing glasses
Written 2021-10-19 by Stefan Harmeling

Based on really nice example from Philipp Hennig, 2014.

In [ ]:
from numpy             import linspace
from scipy.special     import beta as beta_fn  # beta function
from scipy.stats       import beta             # beta distribution
from plotly.subplots   import make_subplots
from plotly.graph_objs import FigureWidget
from ipywidgets        import interact, Button, Text, Layout, HBox

## Beta distribution

In [ ]:
# implement the PDF of the Beta distribution
def beta_pdf(x, a, b):
    return (x**(a-1)) * ((1-x)**(b-1)) / beta_fn(a, b)

In [ ]:
# compare my implementation with the scipy one by plotting it
x  = linspace(0, 1, 100)
a  = 7.0
b  = 3.0
f1 = make_subplots(rows=1, cols=2, shared_yaxes=True,
                   subplot_titles=("beta_pdf(x, a, b)", "scipy.stats.beta.pdf(x, a, b)"))
f1.add_scatter(x=x, y=beta_pdf(x, a, b), row=1, col=1)
f1.add_scatter(x=x, y=beta.pdf(x, a, b), row=1, col=2)
display(f1)

In [ ]:
# interactively show the density
x = linspace(0, 1, 100)
a, b = 1.0, 1.0
f2 = FigureWidget()  # create a figure that can be updated
f2.add_scatter(x=x, y=beta.pdf(x, a, b))
@interact(a=(0.01, 20.0, 0.1), b=(0.01, 20.0, 0.1))
def update(a=1.0, b=1.0):
    s = (f"mean = {a/(a+b)}, "
         f"variance = {a*b/(a+b+1)/((a+b)**2)}")
    f2.update_layout(title={'text': s})
    f2.data[0].y = beta.pdf(x, a, b)
display(f2)

## Wearing glasses demo

The probability of wearing glasses is represented by random variable $Y$, which is `y` in the code. The observations are represented as random variables $X_1$, $X_2$, \ldots  In the implementation uses buttons for the observations.  The counting is done by string-typed `t0.value` and `t1.value`.

In [ ]:
y = linspace(0, 1, 100)     # possible values of RV Y
a_prior, b_prior = 1.0, 1.0 # prior parameters
f3 = FigureWidget()    # create a figure that can be updated
f3.add_scatter(x=y, y=beta.pdf(y, a_prior, b_prior))
def update(t0_inc=False, t1_inc=False, reset=False):
    if t0_inc: t0.value = str(float(t0.value)+1)
    if t1_inc: t1.value = str(float(t1.value)+1)
    if reset:  t0.value, t1.value = "0", "0"
    try:
        b = b_prior + float(t0.value)
    except ValueError:
        b, t0.value, b = b_prior, "0"
    try:
        a = a_prior + float(t1.value)
    except ValueError:
        a, t1.value = a_prior, "0"
    s = (f"b = {b}, a = {a}, "
         f"mean = {a/(a+b)}, "
         f"variance = {a*b/(a+b+1)/((a+b)**2)}")
    f3.update_layout(title={'text': s})
    f3.data[0].y = beta.pdf(y, a, b)
b0 = Button(description='w/o glasses',  layout=Layout(width='20%'))
b1 = Button(description='with glasses', layout=Layout(width='20%'))
re = Button(description='RESET',        layout=Layout(width='40%'))
b0.on_click(lambda dummy: update(t0_inc=True))
b1.on_click(lambda dummy: update(t1_inc=True))
re.on_click(lambda dummy: update(reset=True))
t0 = Text(str(0),                       layout=Layout(width= '8%'))
t1 = Text(str(0),                       layout=Layout(width= '8%'))
t0.on_submit(lambda dummy: update())
t1.on_submit(lambda dummy: update())
update()          # to get the initial title correct
display(f3, HBox([b0, t0, re, t1, b1]))